# Encontros 13 e 14: Datathon

Importando as bibliotecas:

In [ ]:
import sys
sys.path.append('..')

import socnet as sn
import numpy as np

from scipy.stats.stats import pearsonr
from scipy.optimize import minimize

Definindo função que calcula *coreness*:

In [ ]:
def cost(c, A, mask):
    row = np.matrix(c)

    C = np.multiply(row.transpose(), row)

    correlation, _ = pearsonr(C[~mask].flat, A[~mask].flat)

    return -correlation


def calculate_coreness(g):
    length = g.number_of_nodes()

    c = np.random.rand(length)

    A = sn.build_matrix(g)

    mask = np.eye(length, dtype=bool)

    bounds = [(0, 1) for _ in range(length)]

    result = minimize(cost, c, args=(A, mask), bounds=bounds)

    norm = 0

    for n in g.nodes:
        g.nodes[n]['coreness'] = result['x'][n]

        norm += g.nodes[n]['coreness'] ** 2

    for n in g.nodes:
        g.nodes[n]['coreness'] /= norm

Definindo função que calcula *PageRank*:

In [ ]:
def equals(a, b):
    return abs(a - b) < 0.000000001


def calculate_pagerank(g):
    length = g.number_of_nodes()

    k = 10
    scale = 0.8
    residue = (1 - scale) / length

    R = sn.build_matrix(g)

    for n in g.nodes:
        total = np.sum(R[n,])

        if equals(total, 0):
            R[n, n] = 1
        else:
            R[n,] /= total

    R = scale * R + residue

    Rt = R.transpose()

    rank = 1 / length

    r = np.full((length, 1), rank)

    for _ in range(k):
        r = Rt.dot(r)

    for n in g.nodes:
        g.nodes[n]['pagerank'] = r[n, 0]

Lista de todos os códigos de países, para facilitar a iteração pelos arquivos.

Não esqueça que também existem os arquivos `<ano>-all-countries.gml` que representam a soma dos países.

In [ ]:
country_codes = ['all-countries',
    'AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHN', 'CYP',
    'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR',
    'GRC', 'HUN', 'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR',
    'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'POL', 'PRT',
    'ROM', 'RUS', 'SVK', 'SVN', 'SWE', 'TUR', 'TWN', 'USA',
]

Lista de todos os códigos de indústrias, para facilitar a iteração pelos arquivos.

Não esqueça que também existem os arquivos `<ano>-all-industries.gml` que representam a soma das indústrias.

In [ ]:
industry_codes = ['all-industries',
     'c1',  'c2',  'c3',  'c4',  'c5',  'c6',  'c7',
     'c8',  'c9', 'c10', 'c11', 'c12', 'c13', 'c14',
    'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21',
    'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28',
    'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35',
]

file = open('in-out.txt', 'w')

for year in range(1995, 2007):
    for code in industry_codes:
        g = sn.load_graph('grafos/{}-{}.gml'.format(year, code))
        for n in g.nodes:
            ind = sum([g.edges[m, n]['weight'] for m in g.predecessors(n)])
            oud = sum([g.edges[n, m]['weight'] for m in g.successors(n)])
            file.write('{}\t{}\t{}\t{}\t{}\n'.format(year, code, g.nodes[n]['code'], ind, oud))

file.close()